In [ ]:
%load_ext memory_profiler
%load_ext line_profiler

from sim_util import preview_from_definition
from matplotlib import pyplot as plt
import numpy as np
from matplotlib import colors
import gputools
import os
from skimage.io import imsave
import logging

h = logging.StreamHandler()
h.setLevel(logging.INFO)
h.setFormatter(logging.Formatter ('%(asctime)s %(levelname)s:%(message)s'))
logger = logging.getLogger()
logger.addHandler(h)
logger.setLevel(logging.INFO)
%matplotlib inline
plt.rcParams['figure.figsize'] = [16,16]


# set and check which gpu is used
gputools.init_device(id_platform=0, id_device=1)
gputools.get_device()

In [ ]:
# simulate one plane of images
z_plane = 512

# for all parameter files in this dir:
param_basedir = '/scratch/hoerl/sim_preview_test/'
params = [f for f in os.listdir(param_basedir) if f.endswith('.json')]

previews = {}
for p in params:
    print('simulating {}.'.format(p))
    %mprun -f preview_from_definition preview = preview_from_definition(os.path.join(param_basedir, p), z_plane)
    previews[p] = preview
    print('{} done.'.format(p))
print('all done.')

In [ ]:
# plot the planes
# for different lambda, illum, x_pos

for p_file, preview in previews.items():

    for k in sorted(set(map(lambda x: x[::2], preview.keys()))):
        lam, xpos = k
        print(p_file, k)
        k_right = (lam, True, xpos)
        k_left = (lam, False, xpos)
        v_r = preview[k_right]
        v_l = preview[k_left]

        plt.figure()
        sig_r, desc_r = v_r
        sig_l, desc_l = v_l
        
        imsave(os.path.join(param_basedir, p_file + 'sig_l.tif'), sig_l.squeeze().astype(np.float32))
        imsave(os.path.join(param_basedir, p_file + 'sig_r.tif'), sig_r.squeeze().astype(np.float32))
        imsave(os.path.join(param_basedir, p_file + 'desc_l.tif'), desc_l.squeeze().astype(np.float32))
        imsave(os.path.join(param_basedir, p_file + 'desc_r.tif'), desc_r.squeeze().astype(np.float32))
        
        sig_rgb = np.stack((sig_r.squeeze(), sig_l.squeeze(), np.zeros_like(sig_r.squeeze())), 2) 
        # crude intensity scaling
        sig_rgb *= 200
        plt.imshow(sig_rgb.astype(np.float32))
        plt.show()

        

In [ ]:
from sim_util import load_tiff_sequence

img = load_tiff_sequence('/scratch/hoerl/sim_tissue1k/sim-phantom/')

In [ ]:
img.dtype

In [ ]:
import numpy as np
from scipy import ndimage as ndi
from multithreading import Pool, cpu_count
from skimage.data import camera
import os, sys

from matplotlib import pyplot as plt
%matplotlib inline

def apply_mp(arr, fun, overlap, axis=-1, n_proc=None, *args, **kwargs):
    
    if n_proc is None:
        n_proc = cpu_count()        
    
    if axis<0:
        axis = len(arr.shape) + axis
    slice_len = arr.shape[axis] // n_proc
    
    blocks = []
    for i in range(0, n_proc):
        min_ = slice_len * i - overlap
        if i == 0:
            min_ = 0
        
        max_ = slice_len * (i + 1) + overlap
        if i == n_proc-1:
            max_ = arr.shape[axis]
        
        cut = [slice(min_, max_) if d==axis else slice(0,arr.shape[d]) for d in range(len(arr.shape))]
        block = arr[tuple(cut)]
        blocks.append(block)
    
    # FIXME: crude solution, the_fun needs to be module-level for mp
    global the_fun
    def the_fun(a):
        return fun(a, *args, **kwargs)
    
    with Pool(n_proc) as pool:
        res_blocks = pool.map(the_fun, blocks)

    
    res = np.zeros_like(arr)
    for i in range(0, n_proc):
        min_block = overlap if i!=0 else 0
        max_block = res_blocks[i].shape[axis] - (0 if i == (n_proc-1) else overlap)
        min_out = slice_len * i
        max_out = slice_len * (i+1) if i!=(n_proc-1) else arr.shape[axis]
        
        cut_out = [slice(min_out, max_out) if d==axis else slice(0,arr.shape[d]) for d in range(len(arr.shape))]
        cut_block = [slice(min_block, max_block) if d==axis else slice(0,arr.shape[d]) for d in range(len(arr.shape))]
        
        res[tuple(cut_out)] = res_blocks[i][tuple(cut_block)]
    return res

In [ ]:
import os
from scipy.ndimage import gaussian_filter

a = np.random.random((256,1024,1024))

from timeit import timeit
#blr = apply_mp(a, ndi.gaussian_filter, 20, 0, sigma=1)



#%timeit apply_mp(a, ndi.gaussian_filter, 20, 0, sigma=1)
%mprun -f  gaussian_filter gaussian_filter(a, sigma=1)

#plt.imshow(blr)

In [ ]:
import gc
gc.collect()